In [9]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [10]:
import requests
import geocoder

USER = "uyscddawqhg16xwezfqn4wrtnnceww"
API = "asjgx3ytj5xn5gsrqhrosaukzrrb9v"

def send_message():
    g = geocoder.ip('me')
    msg = f"Person may be sleepy at {g.latlng}"
    payload = {"message": msg, "user": USER, "token": API }
    r = requests.post('https://api.pushover.net/1/messages.json', data=payload, headers={'User-Agent': 'Python'})
    return r

In [11]:
from pygame import mixer
mixer.init()
mixer.music.load("./alarm.mp3")
mixer.music.play(-1) 
mixer.music.pause()	 

In [12]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [13]:
thresh = 0.25
frame_check = 5

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

cap=cv2.VideoCapture(0)
flag=0

In [14]:
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor(".\shape_predictor_68_face_landmarks.dat")

In [15]:
while True:
    	
	ret, frame=cap.read()
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	
	subjects = detect(gray, 0)
	
	for subject in subjects:
		shape = predict(gray, subject)
		shape = face_utils.shape_to_np(shape)
		
		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]

		eye = (eye_aspect_ratio(leftEye) + eye_aspect_ratio(rightEye)) / 2.0

		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		
		cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
		
		if eye < thresh:
			flag += 1
			if flag >= frame_check:
    				
				cv2.putText(frame, "****************ALERT!****************", (10, 30),
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
				cv2.putText(frame, "****************ALERT!****************", (10,325),
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
				
				if flag > 60:
					send_message()
				mixer.music.unpause()	 

		else:
			flag = 0
			mixer.music.pause()	 

	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		break
	
cv2.destroyAllWindows()
cap.release()